In [8]:
import pandas as pd
import os
import re

# ==========================================
# 1. CONFIGURACIÓN Y CARGA
# ==========================================
filename = '2025_3T_Flujosporentidadfederativa_orig__11_A1.xlsx'
rutas = [filename, os.path.join("data", "raw", filename), os.path.join("..", "data", "raw", filename)]

file_path = None
for r in rutas:
    if os.path.exists(r):
        file_path = r
        break

if not file_path:
    raise FileNotFoundError(f"❌ No se encontró: {filename}")

print(f"📂 Procesando: {file_path}")

# --- MAPEO DE COLUMNAS ---
df_head = pd.read_excel(file_path, sheet_name='Actividad económica_SCIAN 2023', header=None, nrows=10)

header_idx = None
for idx, row in df_head.iterrows():
    if "Entidad Federativa" in str(row[0]):
        header_idx = idx
        break

if header_idx is None: raise ValueError("❌ Sin encabezados.")

years_row = df_head.iloc[header_idx].tolist()
quarters_row = df_head.iloc[header_idx + 1].tolist()

col_map = {}
current_year = None
for i in range(1, len(years_row)):
    if pd.notna(years_row[i]):
        try:
            y = int(float(years_row[i]))
            if y > 2000: current_year = y
        except: pass
    
    if current_year and pd.notna(quarters_row[i]):
        try:
            q = int(float(quarters_row[i]))
            if 1 <= q <= 4: col_map[i] = (current_year, q)
        except: pass

last_period = max(col_map.values())
prev_period = (last_period[0]-1, last_period[1])
idx_act = [k for k, v in col_map.items() if v == last_period][0]
idx_prev = [k for k, v in col_map.items() if v == prev_period]
idx_prev = idx_prev[0] if idx_prev else None

print(f"📅 Datos extraídos de: {last_period} (Col {idx_act})")

# ==========================================
# 2. EXTRACCIÓN (SOLO 3 DÍGITOS)
# ==========================================
df_data = pd.read_excel(file_path, sheet_name='Actividad económica_SCIAN 2023', header=header_idx+2)

def clean(x):
    if pd.isna(x): return 0.0
    s = str(x).strip().replace(',','')
    try: return float(s)
    except: return 0.0

def clasificar(cod):
    if cod.startswith('1'): return 'Primaria'
    if cod.startswith(('2','3')): return 'Secundaria'
    return 'Terciaria'

filas = []
estado_act = None

for i, row in df_data.iterrows():
    c = str(row.iloc[0]).strip()
    if not c or c == 'nan': continue
    
    match = re.match(r'^(\d{2,6}|31-33)\s+(.*)', c)
    if match:
        cod, desc = match.groups()
        
        # Filtro estricto: Solo 3 dígitos, sin agrupadores
        if len(cod) == 3 and cod != '31-33':
            val_act = clean(row.iloc[idx_act])
            val_prev = clean(row.iloc[idx_prev]) if idx_prev else 0.0
            
            filas.append({
                'Estado': estado_act,
                'Codigo': cod,
                'Actividad': desc,
                'Sector': clasificar(cod),
                'Inversion': val_act,
                'Inversion_Anterior': val_prev
            })
            
    elif not c[0].isdigit() and "total" not in c.lower() and "nota" not in c.lower():
        estado_act = c

df_clean = pd.DataFrame(filas)

# ==========================================
# 3. RESULTADOS (TOP 3 PURO)
# ==========================================
if not df_clean.empty:
    
    tops = []
    for est in df_clean['Estado'].unique():
        d_est = df_clean[df_clean['Estado'] == est]
        for sec in ['Primaria', 'Secundaria', 'Terciaria']:
            d_sec = d_est[d_est['Sector'] == sec]
            
            # Top 3 descendente por Monto
            top3 = d_sec.sort_values('Inversion', ascending=False).head(3).copy()
            
            # SIN CÁLCULO DE PARTICIPACIÓN
            tops.append(top3)
            
    df_final = pd.concat(tops)
    
    # --- VERIFICACIÓN AGUASCALIENTES ---
    print("\n🔎 REVISIÓN AGUASCALIENTES (Top 3 por Flujo):")
    ags = df_final[df_final['Estado'] == 'Aguascalientes']
    
    for sec in ['Primaria', 'Secundaria', 'Terciaria']:
        print(f"\n--- {sec} ---")
        d = ags[ags['Sector'] == sec]
        if not d.empty:
            for _, r in d.iterrows():
                print(f"   > {r['Codigo']} {r['Actividad'][:40]}...: ${r['Inversion']:,.2f} M")
        else:
            print("   (Sin datos)")

    # Totales para referencia (KPI Principal)
    print("\n📊 TOTALES ESTATALES (Suma de Top 3 vs Realidad):")
    # Nota: El KPI principal debe venir de la suma de todo (df_clean), no solo del top 3
    total_ags = df_clean[df_clean['Estado']=='Aguascalientes']['Inversion'].sum()
    print(f"Total Aguascalientes (Suma 3 dígitos): ${total_ags:,.2f} M")

else:
    print("❌ Error: Sin datos.")

📂 Procesando: ..\data\raw\2025_3T_Flujosporentidadfederativa_orig__11_A1.xlsx
📅 Datos extraídos de: (2025, 3) (Col 9)

🔎 REVISIÓN AGUASCALIENTES (Top 3 por Flujo):

--- Primaria ---
   > 111 Agricultura...: $15.06 M
   > 112 Cría y explotación de animales...: $0.00 M
   > 113 Aprovechamiento forestal...: $0.00 M

--- Secundaria ---
   > 336 Fabricación de equipo de transporte...: $328.20 M
   > 333 Fabricación de maquinaria y equipo...: $39.90 M
   > 326 Industria del plástico y del hule...: $12.43 M

--- Terciaria ---
   > 431 Comercio al por mayor de abarrotes, alim...: $52.71 M
   > 462 Comercio al por menor en tiendas de auto...: $6.08 M
   > 488 Servicios relacionados con el transporte...: $4.35 M

📊 TOTALES ESTATALES (Suma de Top 3 vs Realidad):
Total Aguascalientes (Suma 3 dígitos): $200.43 M
